## Events in Cologne

In [4]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [116]:
url="https://www.stadt-koeln.de/leben-in-koeln/veranstaltungen/index.html?from=20190801&to=20211101"
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
calendar = soup.find(id="veranstaltungssuchergebnis")

category=soup.find(id='veranstaltung_kategorie')
#category


cat={}
for option in category.find_all('option'):
    cat[option.text]=option['value']
    #print (option['value'], option.text)


del cat['Alle Kategorien']
cat

{'Ausstellung': '27',
 'Bauleitplanung Öffentlichkeitsbeteiligung': '92',
 'Beratung': '83',
 'Bürgerhaus Kalk': '44',
 'Bürgerhaus Stollwerck': '32',
 'Bürgerzentrum Chorweiler': '91',
 'Dezernat III': '90',
 'Digital': '88',
 'Diskussion': '85',
 'Existenzgründung': '49',
 'Familie': '75',
 'Ferienprogramm': '28',
 'Film': '46',
 'Frauen + Gleichstellung': '45',
 'Führung': '23',
 'Geschichte': '52',
 'Gesundheit': '89',
 'Historisches Archiv': '71',
 'Interkulturelle Woche': '43',
 'Internationales': '80',
 'Kabarett': '15',
 'Kinder + Jugend': '21',
 'Kleinkunst': '13',
 'Konzert': '7',
 'Kultur': '99',
 'Lesung': '37',
 'Literatur': '53',
 'Menschen mit Behinderungen': '60',
 'Messen': '94',
 'Museum': '50',
 'Natur und Grün': '17',
 'Planen und Bauen': '74',
 'Radfahren': '96',
 'Rathaussaal Porz': '101',
 'Rheinische Musikschule': '14',
 'Seniorinnen und Senioren': '34',
 'Sonstige Veranstaltungen': '12',
 'Sport': '64',
 'Stadtbibliothek': '59',
 'Städtepartnerschaft': '42',
 '

In [144]:


## Calendars 
# Kategorie=94 e.g. Messen
cat_element='101'

def get_search_size(cat_element):
    url=''.join(["https://www.stadt-koeln.de/leben-in-koeln/veranstaltungen/index.html?from=20190801&to=20211101&veranstaltung_kategorie=",cat_element])
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')

    event_nu = soup.find(class_="innerer_block1 grid_1 firstchild")
    if event_nu is not None:
        len_=int(event_nu.get_text().split(' ')[-1])
    else:
        len_=0
    #try:
        #event_nu = soup.find(class_="innerer_block1 grid_1 firstchild")
        #print(event_nu.get_text())

        #len_=int(event_nu.get_text().split(' ')[-1])
        #print(len_)
    #except: len_=2640
    print(len_)
    return url,len_
    



In [145]:
url,len_=get_search_size(cat_element)

Treffer 1–10 von 355
355


In [146]:
def get_event_list(url,len_):
    event_list=list()
    len_=len_ - (len_%10)

    while len_>=0:

        #print(len_)
        string=''.join(['&start=',str(len_),'#'])
        #print(string)
        url2=''.join([url,string])
        #print(url2)
        page = requests.get(url2)
        soup = BeautifulSoup(page.content, 'html.parser')
        calendar = soup.find(id="veranstaltungssuchergebnis")


        for a in calendar.find_all('a', href=True):
            event_list.append( ''.join(['https://www.stadt-koeln.de', a['href']]))
        len_=len_-10
    return event_list

In [148]:
len(lst)

355

In [149]:
# populate df with every event


def populate_df(lst):
    e_df=pd.DataFrame()
    for url in lst:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        try:
            address=soup.find(itemprop="address").get_text().strip("'\n'")
            if address=='Online-Veranstaltung': continue
        
            e_df.loc[url,'address']=address
        except:continue
            
        start_date=soup.find(itemprop="startDate").get('content')

        end_date=soup.find(itemprop="endDate").get('content')

        e_df.loc[url,'start_date']=start_date
        e_df.loc[url,'end_date']=end_date

        
        description=soup.find(itemprop="description").get_text().strip("'\n'")

        e_df.loc[url,'description']=description


        time=soup.find(itemprop="OpeningHoursSpecification")
        #print(time)
        if time is not None:

            time=time.get_text().strip("'\n'")
            e_df.loc[url,'time']=time
            #print(time)

        for tag in soup.find_all("meta"):
            if tag.get("name")=='geo.position':
                geo=tag.get('content')
                e_df.loc[url,'latitude']=geo.split(',')[0]
                e_df.loc[url,'longitude']=geo.split(',')[1]

    return e_df


In [140]:
#e_df=pd.DataFrame()
for k,cat_element in cat.items():
    print(k,cat_element)
    url,len_=get_search_size(cat_element)
    lst=get_event_list(url,len_)
    print(len(lst))
    if len(lst)>0:
        df=populate_df(lst)
        df['event_type']=k
    e_df=pd.concat([e_df,df])

Ausstellung 86
Treffer 1–10 von 74
74
Bauleitplanung Öffentlichkeitsbeteiligung 86
Treffer 1–10 von 82
82
Beratung 86
Treffer 1–10 von 37
37
Bürgerhaus Kalk 86
Treffer 1–10 von 33
33
Bürgerhaus Stollwerck 86
Treffer 1–10 von 235
235
Bürgerzentrum Chorweiler 86
Treffer 1–9 von 9
9
Dezernat III 86
Treffer 1–10 von 22
22
Digital 86
Treffer 1–10 von 125
125
Diskussion 86
Treffer 1–10 von 90
90
Existenzgründung 86
Treffer 1–10 von 33
33
Familie 86
Treffer 1–10 von 198
198
Ferienprogramm 86
Treffer 1–10 von 635
635
Film 86
Treffer 1–10 von 56
56
Frauen + Gleichstellung 86
Treffer 1–10 von 40
20
Führung 86
Treffer 1–10 von 104
84
Geschichte 86
Treffer 1–10 von 42
42
Gesundheit 86
Treffer 1–10 von 38
38
Historisches Archiv 86
Treffer 1–10 von 36
36
Interkulturelle Woche 86
Treffer 1–10 von 34
34
Internationales 86
Treffer 1–10 von 42
42
Kabarett 86
Treffer 1–10 von 241
241
Kinder + Jugend 86
Treffer 1–10 von 959
919
Kleinkunst 86
Treffer 1–10 von 250
250
Konzert 86
Treffer 1–10 von 236
236
Kul

AttributeError: 'NoneType' object has no attribute 'get_text'

In [157]:
e_df.shape

(4137, 8)

In [158]:
e_df.tail(2)

,address,start_date,end_date,description,time,latitude,longitude,event_type
https://www.stadt-koeln.de/leben-in-koeln/freizeit-natur-sport/veranstaltungskalender/fahrradbeauftragter-tour-poll,Marktplatz Poll\n\nSiegburger Straße/Rolshover...,2019-10-18,2019-10-18,Wir sind zu Gast in Poll: Während des Wochenma...,10 bis 14 Uhr,50.91545929128713,6.993565535937478,Radfahren
https://www.stadt-koeln.de/leben-in-koeln/freizeit-natur-sport/veranstaltungskalender/fahrradtour-zum-benrather-schloss,Diverse Veranstaltungsorte in\nKöln,2020-04-18,2020-04-18,Die Veranstaltung findet zu diesem Zeitpunkt n...,10 Uhr,50.938108,6.95905,Radfahren


In [159]:
for k in list(cat)[33:]:
    print(k,cat[k])

Rathaussaal Porz 101
Rheinische Musikschule 14
Seniorinnen und Senioren 34
Sonstige Veranstaltungen 12
Sport 64
Stadtbibliothek 59
Städtepartnerschaft 42
Tagung 57
Tanz 84
Theater 4
Umwelt und Grün 9
Verkehr 78
Versteigerung 38
Vortrag 6
Weiterbildung 54
Wissenschaft im Rathaus 68
Workshop 86


In [160]:
for k in list(cat)[33:]:
    print(k,cat[k])
    cat_element=cat[k]
    url,len_=get_search_size(cat_element)
    lst=get_event_list(url,len_)
    #print(len(lst))
    if len(lst)>0:
        df=populate_df(lst)
        df['event_type']=k
    e_df=pd.concat([e_df,df])

Rathaussaal Porz 101
0
Rheinische Musikschule 14
Treffer 1–10 von 99
99
99
Seniorinnen und Senioren 34
Treffer 1–10 von 74
74
74
Sonstige Veranstaltungen 12
Treffer 1–10 von 202
202
202
Sport 64
Treffer 1–10 von 180
180
160
Stadtbibliothek 59
Treffer 1–10 von 964
964
964
Städtepartnerschaft 42
Treffer 1–10 von 44
44
44
Tagung 57
Treffer 1–4 von 4
4
4
Tanz 84
Treffer 1–10 von 23
23
23
Theater 4
Treffer 1–10 von 259
259
259
Umwelt und Grün 9
Treffer 1–10 von 92
92
92
Verkehr 78
Treffer 1–10 von 50
50
50
Versteigerung 38
Treffer 1–1 von 1
1
1
Vortrag 6
Treffer 1–10 von 214
214
214
Weiterbildung 54
Treffer 1–10 von 138
138
138
Wissenschaft im Rathaus 68
Treffer 1–10 von 27
27
27
Workshop 86
Treffer 1–10 von 355
355
355


In [161]:
e_df.shape

(6646, 8)

In [163]:
for i, row in e_df[e_df['start_date']==e_df['end_date']].iterrows():
    try:
        e_df.loc[i,'start_time']=row['time'].split('bis')[0]
        e_df.loc[i,'end_time']=row['time'].split('bis')[1].split('Uhr')[0]
    except: continue


In [164]:
e_df.tail(2)

,address,start_date,end_date,description,time,latitude,longitude,event_type,start_time,end_time
https://www.stadt-koeln.de/leben-in-koeln/freizeit-natur-sport/veranstaltungskalender/makerspace-roboter-nao-einfuehrung-die-programmierung-2,Stadtbibliothek Köln\n\nJosef-Haubrich-Hof 1\n...,2019-09-14,2019-09-14,Wie arbeite ich mit einem humanoiden Roboter? ...,11 bis 13 Uhr,50.938108,6.95905,Workshop,11,13
https://www.stadt-koeln.de/leben-in-koeln/freizeit-natur-sport/veranstaltungskalender/20-improvisiakum-1,Rheinische Musikschule - Regionalschule Ehrenf...,2019-09-13,2019-09-15,Beschreiten Sie an insgesamt drei Tagen neue W...,"Eröffnungskonzert: \n13. September, 20 Uhr\n\n...",50.9441504,6.923948699999983,Workshop,NaN,NaN


In [28]:
#e_df.to_csv('events.csv')